# Analysis 3

In [75]:
import argparse
import json
import pprint
import requests
import sys
import urllib
import csv
import time
import datetime
import pandas as pd
import glob
from pandas.io.json import json_normalize
from numpy import nan as NA
import nltk
import operator
from nltk.probability import FreqDist
from nltk.corpus import stopwords

from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

In [2]:
CLIENT_ID = 'gBcvfFpabk3CnzmXKdYe3w'
CLIENT_SECRET = 'K9dIltpo1DH4hTrKy6c7YhxR2bwENElpzqTEaRz44QXxKnImd3cSOnYmhrucVHns'
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
TOKEN_PATH = '/oauth2/token'
GRANT_TYPE = 'client_credentials'
SEARCH_LIMIT = 50

In [3]:
def obtaintoken(host, path):
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    assert CLIENT_ID, "Please supply your client_id."
    assert CLIENT_SECRET, "Please supply your client_secret."
    data = urlencode({
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': GRANT_TYPE,
    })
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }
    response = requests.request('POST', url, data=data, headers=headers)
    token = response.json()['access_token']
    return token

token = obtaintoken(API_HOST, TOKEN_PATH)
token


'Z1C_ixTGccfADwGnwzG5yUE_tqYbJ3Vfj33NrA5Qi_LhXdj_A0idBPo8mP2dHU71s6HAkJuCJDxhH-BJ1WSDno-pyfmit-GNuVB8gPlu2kxjw-tgGZSO6VHnG4H_WHYx'

In [4]:
def request(host, path, token, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % token,
    }

#     pr\int(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

def search(token, term, location):
    

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, token, url_params=url_params)

def get_business(token, business_id):
    
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, token)

def query_api(term, location):
   

    response = search(token, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(token, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)


In [35]:
def findresult(categories):
    page = 0
    url_params = {
                'location': 'New York, NY',
                'limit': SEARCH_LIMIT,
                'sort_by':'review_count',
                'categories': categories,
                'offset':0
            }
    response1 = request(API_HOST, SEARCH_PATH, token, url_params=url_params)
    with open('data/foodcompare/'+ categories + str(page) +'.json', 'a') as outfile:
        json.dump(response1, outfile)

    while page < (response1['total']/50) and (page < 19):
        page = page + 1
        url_params = {
                'location': 'New York, NY',
                'limit': SEARCH_LIMIT,
                'sort_by':'review_count',
                'categories': categories,
                'offset':50*page

            }

        response = request(API_HOST, SEARCH_PATH, token, url_params=url_params)
        with open('data/foodcompare/' + categories + str(page) +'.json', 'a') as outfile:
            json.dump(response, outfile)

In [36]:
Chinese ='chinese'
Mexican = 'mexican'
French = 'french'
findresult(Chinese)
findresult(Mexican)
findresult(French)

In [38]:
pathList = []
for filepath in glob.glob('/Users/xiongyiming/Documents/INFO7374Python/Final Project/data/foodcompare/*.json'):
    pathList.append(filepath)    

In [39]:
dfch=pd.DataFrame()
dfme=pd.DataFrame()
dffr=pd.DataFrame()

for file in pathList:
    if 'chinese' in file:
        
        with open(file) as data_file:    
            datajson = json.load(data_file)
            if 'businesses' in datajson:
                df1 = json_normalize(datajson['businesses'])
                df1 = df1.reindex(columns=['name', 'price', 'rating', 'review_count','id'])
                df2 = df1[['name', 'price', 'rating', 'review_count','id']]
                dfch = pd.concat([dfch,df2])
                
    if 'mexican' in file:
        with open(file) as data_file:    
            datajson = json.load(data_file)
            if 'businesses' in datajson:
                df1 = json_normalize(datajson['businesses'])
                df1 = df1.reindex(columns=['name', 'price', 'rating', 'review_count','id'])
                df2 = df1[['name', 'price', 'rating', 'review_count','id']]
                dfme = pd.concat([dfme,df2])
                
    if 'french' in file:
        with open(file) as data_file:    
            datajson = json.load(data_file)
            if 'businesses' in datajson:
                df1 = json_normalize(datajson['businesses'])
                df1 = df1.reindex(columns=['name', 'price', 'rating', 'review_count','id'])
                df2 = df1[['name', 'price', 'rating', 'review_count','id']]
                dffr = pd.concat([dffr,df2])
                
    

In [67]:
def findid():
    businessidlist = []
    i = 0
    while i < (len(dfch) -1):
        businessid = str(dfch.iloc[i]['id'])
        businessidlist.append(businessid)
        i+=1
    return businessidlist

businessidlist= findid()
# businessidlist

In [58]:
def getreview(businessid):
    business_path = BUSINESS_PATH + businessid + '/reviews'
    return request(API_HOST, business_path, token)

In [69]:
worddata = getreview(businessid)
worddata

{'reviews': [{'rating': 4,
   'text': 'When you walk into the restaurant tucked away in a Chinatown alley, you\'ll immediately be greeted by a newspaper article on the door proclaiming "Joe\'s" to...',
   'time_created': '2017-04-25 17:12:52',
   'url': 'https://www.yelp.com/biz/joes-shanghai-new-york-2?hrid=iw77oOcLTIvrx8wHnuIkyg&adjust_creative=gBcvfFpabk3CnzmXKdYe3w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=gBcvfFpabk3CnzmXKdYe3w',
   'user': {'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/W3-KILg-L07S2k46-9iPWw/o.jpg',
    'name': 'Sherry X.'}},
  {'rating': 5,
   'text': "Soooo annoying to heap on another glowing review for Joe's xiao long bao, but when you make the absolute best soup dumplings, you deserve all the...",
   'time_created': '2017-04-25 13:44:13',
   'url': 'https://www.yelp.com/biz/joes-shanghai-new-york-2?hrid=23iJA0wIC7bLJlH8PgDb1w&adjust_creative=gBcvfFpabk3CnzmXKdYe3w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&

In [78]:
wordlist=[]

for i in worddata['reviews']:
    for word in i['text'].split():
            if word.isalpha():
                word = word.lower()
                if word not in stopwords.words('english'):         
                    wordlist.append(word)
    
#     wordlist.append(i['text'])

In [79]:
wordlist

['walk',
 'restaurant',
 'tucked',
 'away',
 'chinatown',
 'immediately',
 'greeted',
 'newspaper',
 'article',
 'door',
 'proclaiming',
 'soooo',
 'annoying',
 'heap',
 'another',
 'glowing',
 'review',
 'xiao',
 'long',
 'make',
 'absolute',
 'best',
 'soup',
 'deserve',
 'believe',
 'spicy',
 'beef',
 'tendon',
 'soup',
 'extra',
 'chilli',
 'sauce',
 'spice',
 'got',
 'best',
 'soup',
 'ever']

In [82]:

                    
result = FreqDist(wordlist) 

sorted_wordlist = sorted(result.items(), key=operator.itemgetter(1))
sorted_wordlist.reverse()
sorted_wordlist[:3]

    

[('soup', 3), ('best', 2), ('review', 1)]